In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

import urllib.request
import pandas as pd
import numpy as np
import time
import ktrain
from ktrain import tabular
from sklearn.model_selection import train_test_split
from tqdm import *
pd.set_option('display.max_columns', None)

from benchmark_circuits import *
import random
import pandas as pd
from tqdm import *
import pkgutil
import warnings
import exrex
import math
import time
import json
import scipy.stats as stats
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from statsmodels.stats.gof import chisquare
import multiprocessing
from sklearn.metrics import precision_recall_fscore_support
warnings.filterwarnings('ignore')


# In[2]:


load_circuits()


# # Select CUTs

# In[3]:


backends = [('FakeAlmaden', 20), ('FakeBoeblingen', 20), ('FakeBrooklyn', 65), ('FakeCairo', 27), ('FakeCambridge', 28), ('FakeCambridgeAlternativeBasis', 28), ('FakeCasablanca', 7), ('FakeGuadalupe', 16), ('FakeHanoi', 27), ('FakeJakarta', 7), ('FakeJohannesburg', 20), ('FakeKolkata', 27), ('FakeLagos', 7), ('FakeManhattan', 65), ('FakeMontreal', 27), ('FakeMumbai', 27), ('FakeNairobi', 7), ('FakeParis', 27), ('FakeRochester', 53), ('FakeSingapore', 20), ('FakeSydney', 27), ('FakeToronto', 27), ('FakeWashington', 127)]
BaselineCircuits,CUTs = train_test_split(get_all_circuits(),train_size=0.2,random_state=13)
BaselineCircuits,CUTs


# # Define QuCAT Oracles

# In[27]:


import rpy2.robjects as robjects
r = robjects.r
r['source']('chisquare.R')

def Uof(observed,expected):
    if len(observed.keys())<1:
        return "F"
    for k in observed.keys():
        if k not in expected.keys():
            return "F"
    return "P"
    
def Wodf(observed,expected):
    test = robjects.globalenv['chisquare']
    try:
        if len(observed)==1 and len(expected)==1:
            return "P"
        
        obs = []
        exp = []
        expected = dict(sorted(expected.items(), key=lambda item: item[0]))
        observed = dict(sorted(observed.items(), key=lambda item: item[0]))
        for k in set(observed.keys()).intersection(expected.keys()):
            obs.append(observed[k])
            exp.append(expected[k])
        
        for k in set(expected.keys()).difference(observed.keys()):
            exp.append(expected[k])
        
        if len(obs)<len(exp):
#             epsilon = 1024-sum(obs)
#             try:
#                 epsilon = epsilon/(len(exp)-len(obs))
#             except:
#                 epsilon = 0
            obs.extend([0 for t in range(len(exp)-len(obs))])
        
        #obs = [int(o*100) for o in obs]
        
        df_result_r = test(robjects.FloatVector(obs),robjects.FloatVector(exp))
        p = np.array(df_result_r)[0]
        #print("expected_o:",expected)
        #print("observed_o:",observed)
        #print("expected:",exp)
        #print("observed:",obs)
        #print("p-value",p)
        if p<0.01:
            return "F"
        else:
            return "P"
    except Exception as e:
        print(e)
        return "F"
    
def convertNaQp2QuCAT_notation(output,value="prob"):
    program_specification = {}
    for x in output["probability"]:
        program_specification[x["bin"]] = x[value]
    return program_specification

def filter_output(output,model,count=True):
    prediction_output = {}
    for state in output["probability"]:
        all_other_probs = sum([x["prob"] for x in output["probability"] if x["bin"]!=state["bin"]])
        odr = state["odds"]
        pos = state["prob"]
        pof = 1-pos
        df = pd.DataFrame([[pof,odr,pos]],columns=["POF","ODR","POS"])
        prediction = model.predict(df)[0][0]
        prediction_output[state["bin"]] = [state["count"],prediction]
        
    filtered_output = {}
    # clamp output to 0 and 1
    for k in prediction_output.keys():
        if prediction_output[k][1]>1:
            if np.abs(prediction_output[k][1]-1)<0.2:
                prediction_output[k][1] = min(prediction_output[k][1],1) 
                filtered_output[k] = prediction_output[k]
            else:
                if prediction_output[k][1]>2:
                    prediction_output[k][1] = prediction_output[k][1]%1
                    filtered_output[k] = prediction_output[k]
                
        elif prediction_output[k][1]<0:
            prediction_output[k][1] = 0.0
            filtered_output[k] = prediction_output[k]
        else:
            filtered_output[k] = prediction_output[k]

    # check for irrelevent values
    if len(filtered_output.keys())>2:
        temp = {}
        maxvalue = max([filtered_output[k][0] for k in filtered_output.keys()])
        for k in filtered_output.keys():
            if (filtered_output[k][0]/maxvalue)>=0.35:
                temp[k] = filtered_output[k]
        filtered_output =  temp
    
    temp1 = {}
    total = sum([filtered_output[k][0] for k in filtered_output.keys()])
    for k in filtered_output.keys():
        if count:
            temp1[k] = int(filtered_output[k][1]*1024)#int(filtered_output[k][1]*total)
        else:
            temp1[k] = filtered_output[k][1]
    filtered_output =  temp1
        
            
    return filtered_output,prediction_output

def prediction_function(predictor,df,return_proba=False,multilabel=False):
    tseq = predictor.preproc.preprocess_test(df, verbose=0)
    tseq.batch_size = predictor.batch_size
    preds = predictor.model.predict(tseq, verbose=0)
    result = (
        preds
        if return_proba or multilabel or not predictor.c
        else [predictor.c[np.argmax(pred)] for pred in preds]
    )
    if multilabel and not return_proba:
        result = [list(zip(predictor.c, r)) for r in result]
    return result

def filter_output2(outputs,model,count=True):
    df = pd.DataFrame(columns=["POF","ODR","POS","INP"])
    for inp in outputs.keys():
        output = outputs[inp]
        for state in output["probability"]:
            all_other_probs = sum([x["prob"] for x in output["probability"] if x["bin"]!=state["bin"]])
            odr = state["odds"]
            pos = state["prob"]
            pof = 1-pos
            df = df.append({"POF":pof,"POS":pos,"ODR":odr,"INP":inp},ignore_index=True)
    
    Prediction = prediction_function(model,df[["POF","ODR","POS"]])
    total_prediction_outputs = {}
    for inp in outputs.keys():
        output = outputs[inp]
        inp_prediction = Prediction[df[df["INP"]==inp].index]
        prediction_output = {}
        for state,prediction in zip(output["probability"],inp_prediction):
            prediction_output[state["bin"]] = [state["count"],prediction[0]]
        #total_prediction_outputs[inp] = prediction_output
    
        filtered_output = {}
        # clamp output to 0 and 1
        for k in prediction_output.keys():
            if prediction_output[k][1]>1:
                if np.abs(prediction_output[k][1]-1)<0.2:
                    prediction_output[k][1] = min(prediction_output[k][1],1) 
                    filtered_output[k] = prediction_output[k]
                else:
                    if prediction_output[k][1]>2:
                        prediction_output[k][1] = prediction_output[k][1]%1
                        filtered_output[k] = prediction_output[k]

            elif prediction_output[k][1]<0:
                prediction_output[k][1] = 0.0
                filtered_output[k] = prediction_output[k]
            else:
                filtered_output[k] = prediction_output[k]

        #check for irrelevent values
        if len(filtered_output.keys())>2:
            temp = {}
            maxvalue = max([filtered_output[k][0] for k in filtered_output.keys()])
            for k in filtered_output.keys():
                if (filtered_output[k][0]/maxvalue)>=0.35:
                    temp[k] = filtered_output[k]
            filtered_output =  temp

        temp1 = {}
        total = sum([filtered_output[k][0] for k in filtered_output.keys()])
        for k in filtered_output.keys():
            if count:
                temp1[k] = int(filtered_output[k][1]*1024)#int(filtered_output[k][1]*total)
            else:
                temp1[k] = filtered_output[k][1]
        filtered_output =  temp1
        
        total_prediction_outputs[inp] = (filtered_output,prediction_output)
        
    return total_prediction_outputs



def oracle(predicted,groundtruth):
    result = Uof(predicted,groundtruth)
    if result=="P":
        result = Wodf(predicted,groundtruth)
        if result=="P":
            return 1
        else:
            return 0
    else:
        return 0
    
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
            TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
            FP += 1
        if y_actual[i]==y_hat[i]==0:
            TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
            FN += 1

    return(TP, FP, TN, FN)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 434.72it/s]


In [2]:
backend_factory = BackendFactory()
backend = backend_factory.initialize_backend()

In [3]:
for cut in tqdm(CUTs[300:500]):
    
    groundTruth = []
    predicted = []
    
    algo = get_circuit_class_object(cut)
    inps = algo.get_full_inputs()
    PS_t = algo.get_result(backend,inps)
    mutant = get_circuit_class_object_mutation(cut+"_M1")
    mutant1ps = mutant.get_result(backend,inps)
    mutant = get_circuit_class_object_mutation(cut+"_M2")
    mutant2ps = mutant.get_result(backend,inps)
    mutant = get_circuit_class_object_mutation(cut+"_M3")
    mutant3ps = mutant.get_result(backend,inps)
    
    for bk,_ in backends:
        
        ndata = open("./generated_data/{}_P{}.json".format(bk,algo.key_aurguments["ID"]),"r")
        Noise_inp = json.load(ndata)
        ndata.close()
        
        ndata = open("./generated_data/{}_P{}_M1.json".format(bk,algo.key_aurguments["ID"]),"r")
        mutant1_Noise_inp = json.load(ndata)
        ndata.close()
        
        ndata = open("./generated_data/{}_P{}_M2.json".format(bk,algo.key_aurguments["ID"]),"r")
        mutant2_Noise_inp = json.load(ndata)
        ndata.close()
        
        ndata = open("./generated_data/{}_P{}_M3.json".format(bk,algo.key_aurguments["ID"]),"r")
        mutant3_Noise_inp = json.load(ndata)
        ndata.close()
        
        predictor = ktrain.load_predictor('tunning_models/{}_{}'.format(bk,cut))
        Noise_inp_predicitons = filter_output2(Noise_inp,predictor,False)
        mutant1_noise_predicitons = filter_output2(mutant1_Noise_inp,predictor,False)
        mutant2_noise_predicitons = filter_output2(mutant1_Noise_inp,predictor,False)
        mutant3_noise_predicitons = filter_output2(mutant1_Noise_inp,predictor,False)
        
        
        for inp in inps:
            ps = PS_t[inp]
            ps_noise = Noise_inp[inp]
            ps = convertNaQp2QuCAT_notation(ps,value='prob')
            ps_noise, _prediction =  Noise_inp_predicitons[inp]

            groundTruth.append(oracle(ps,ps))
            predicted.append(oracle(ps_noise,ps))

            mutantps = mutant1ps[inp] 
            ps_noise = mutant1_Noise_inp
            mutantps = convertNaQp2QuCAT_notation(mutantps,value='prob')
            ps_noise, _prediction =  mutant1_noise_predicitons[inp]

            groundTruth.append(oracle(mutantps,ps))
            predicted.append(oracle(ps_noise,ps))
            
            mutantps = mutant2ps[inp] 
            ps_noise = mutant2_Noise_inp
            mutantps = convertNaQp2QuCAT_notation(mutantps,value='prob')
            ps_noise, _prediction =  mutant2_noise_predicitons[inp]

            groundTruth.append(oracle(mutantps,ps))
            predicted.append(oracle(ps_noise,ps))
            
            mutantps = mutant3ps[inp] 
            ps_noise = mutant3_Noise_inp
            mutantps = convertNaQp2QuCAT_notation(mutantps,value='prob')
            ps_noise, _prediction =  mutant3_noise_predicitons[inp]

            groundTruth.append(oracle(mutantps,ps))
            predicted.append(oracle(ps_noise,ps))

    
    TP,FP,TN,FN = perf_measure(groundTruth,predicted)
    precision,recall,f1,_ = precision_recall_fscore_support(groundTruth,predicted,average="binary")
    with open(f"results/f1/F1_filtered_{cut}.json","w") as file:
        json.dump({"ground":groundTruth,"predicted":predicted},file)

  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0

['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0

  0%|▍                                                                                 | 1/200 [00:17<57:52, 17.45s/it]

['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0

...

['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0

100%|█████████████████████████████████████████████████████████████████████████████▌| 199/200 [1:31:19<00:25, 25.22s/it]

['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0

['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
['Target Value']
   Target Value
0           0.0
1           0.0
2           0.0
3           0.0

100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [1:31:49<00:00, 27.55s/it]


In [4]:
print("Filter")
print("TP = {}, FP = {}, FN = {}, P = {}, R = {}, F1 = {}".format(TP,FP,FN,precision,recall,f1))

Filter
TP = 331, FP = 18, FN = 175, P = 0.9484240687679083, R = 0.6541501976284585, F1 = 0.7742690058479531
